TCA & Glycolysis Flux

In [ ]:
import pandas as pd
import pathlib
import incawrapper
from incawrapper import run_inca
import ast
PROJECT_DIR = pathlib.Path().cwd().parents[1].resolve()
data_folder = PROJECT_DIR / pathlib.Path("C:/Users/HP/Desktop/Master's Project/INCAWrapper")

In [ ]:
tracers_data = pd.read_csv(data_folder / "tracers2.csv",
   converters={'atom_mdv':ast.literal_eval, 'atom_ids':ast.literal_eval} # a trick to read lists from csv
).query("experiment_id == 'exp1'") # we only simulated experiment 1

reactions_data = pd.read_csv(data_folder / "updated tca_gly model.csv")
ms_data = pd.read_csv(data_folder / "TCA_Glycolysis_ms file.csv",
  converters={'labelled_atom_ids': ast.literal_eval} # a trick to read lists from csv
)


In [ ]:
output_file = PROJECT_DIR / pathlib.Path("C:/Users/HP/Desktop/Master's Project/INCAWrapper/TCA_Gly_flux_escher.mat")
script = incawrapper.create_inca_script_from_data(
    reactions_data=reactions_data,
    tracer_data=tracers_data,
    ms_measurements=ms_data,
    experiment_ids=["exp1"]
)
script.add_to_block("options",
    incawrapper.define_options(
        fit_starts=10, # Number of flux estimation restarts
        sim_na=False, # Do not simulate natural abundance
        sim_ss=False # Do INST 13C MFA
    )
)
script.add_to_block("runner", incawrapper.define_runner(output_file, run_estimate=True, run_continuation=True))

In [ ]:
import os

# Change working directory
os.chdir(r"C:/Users/HP/Desktop/Master's Project/INCAWrapper")

# Verify the change
print("Current working directory:", os.getcwd())


In [ ]:
# Write the INCA script to a file
script_filename = "TCA_Gly_Flux_escher.m"
script.save_script(PROJECT_DIR / script_filename)
print(f"INCA script saved to {PROJECT_DIR / script_filename}.")

Go to MATLAB:

cd 'C:/Program Files/INCA/inca2.4'
startup
setpath
inca_script

In [ ]:
res = incawrapper.INCAResults(output_file)

In [ ]:
res.fitdata.get_goodness_of_fit()

In [ ]:
estimated_fluxes = res.fitdata.fitted_parameters.query("type.str.contains('flux')")
estimated_fluxes

In [ ]:
#Save the results to csv
estimated_fluxes.to_csv("estimated_fluxes.csv", index=False)


In [ ]:
import pandas as pd

# Read specific columns of csv file 
df = pd.read_csv("estimated_fluxes.csv", usecols=['id', 'val'])

# Remove 0 values
df_filtered = df[df['val'] != 0].copy()

# Remove 'exch' extension from the id names and strip whitespace
df_filtered['id'] = df_filtered['id'].str.replace(' exch', '').str.strip()

# Set 'id' column as the index
df_filtered.set_index('id', inplace=True)

# Select only the 'val' column as a Series, then save
df_filtered['val'].to_json('output.json', orient='index', indent=2)
